# Partial least squares regression "by hand"

Here, we will find the latent variables in partial least squares (PLS) regression by hand. As an
example we will consider the solubility data again but in reduced form:

* We will only consider alcohols

* We use only logP and the number of atoms in the molecule as our X

* We use the solubility and the molecular weight as our Y

From the regression we did on the full data set, we know that logP is important for predicting the
solubility, and we also expect that the molecular weight is correlated with the number of atoms.

## Loading the data

The data we will use can be found in the file [solubility_alc.csv](./solubility_alc.csv):

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.linalg import svd
from numpy.linalg import norm


%matplotlib notebook
sns.set_theme(style="ticks", context="notebook", palette="muted")

In [ ]:
data = pd.read_csv("solubility_alc.csv")
data

In [ ]:
xvars = ["MolLogP", "nAtom"]
yvars = ["solubility (mol/L)", "Molecular Weight"]

scaler_x, scaler_y = StandardScaler(), StandardScaler()

Y = scaler_y.fit_transform(data[yvars].to_numpy())
X = scaler_x.fit_transform(data[xvars].to_numpy())

### Plotting X- and Y-data

Before we start, let us first plot the X- and Y-data:

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, ncols=2, figsize=(8, 4))
axes[0].scatter(X[:, 0], X[:, 1])
axes[0].set(xlabel=xvars[0], ylabel=xvars[1])
axes[0].set_title("X-data", loc="left")

axes[1].scatter(Y[:, 0], Y[:, 1])
axes[1].set(xlabel=yvars[0], ylabel=yvars[1])
axes[1].set_title("Y-data", loc="left")

sns.despine(fig=fig)

## Finding a latent variable for PLS


In PLS, we are looking for scores $\mathbf{t}$ (for X) and $\mathbf{u}$ (for y) so that the covariance
is maximized. The main idea is that the scores should explain the variance in X, Y, and the
covariance between X and Y.

The covariance can be calculated by $\mathbf{t}^\top \mathbf{u}$ (the dot product between
the scores). Further, the scores are calculated using the *weights* $\mathbf{w}_x$ (for X) and $\mathbf{w}_y$ (for Y):


\begin{align}
\mathbf{t} &= \mathbf{X} \mathbf{w}_x \\
\mathbf{u} &= \mathbf{Y} \mathbf{w}_y
\end{align}


Note that we refer to $\mathbf{w}_x$ and $\mathbf{w}_y$ as *weights*. In PCA, we can invert the relation
giving the scores by multiplying with the transpose of the *loadings*.
In PLS, this is no longer valid for the weights, and PLS introduces
separate *loadings* to do this. For instance, the loadings $\mathbf{p}$ for X so that we can write $\mathbf{X} = \mathbf{t} \mathbf{p}^\top$.

What we will do now, is to create some $\mathbf{w}_x$ and $\mathbf{w}_y$ vectors by hand, and check what
the correlation is (by calculating the correlation and by plotting the scores $\mathbf{t}$ vs. $\mathbf{u}$).
Both $\mathbf{X}$ and $\mathbf{Y}$ have
dimensions $n \times 2$ (we have $n$ samples and $2$ variables). So if $\mathbf{w}_x$ and $\mathbf{w}_y$
have dimensions $2 \times 1$ (that is, they are *column* vectors), then the scores will
have a dimension of $(n \times 2) \times (2 \times 1) = n \times 1$.

## A short example

We set set $\mathbf{w}_x = (0.0, 1.0)^\top$ and $\mathbf{w}_y = (0.0, 1.0)^\top$. Then 
the product $\mathbf{X} \mathbf{w}_x$ will just pick out
the second column of $\mathbf{X}$ (just the number of atoms) and $\mathbf{Y} \mathbf{w}_y$ will
pick out the second column of $\mathbf{Y}$ (just the molecular weight). Then a plot of
$\mathbf{t}$ vs. $\mathbf{q}$ will show how the number of atoms is correlated with the
molecular weight.

In [ ]:
def make_plot(X, Y, wx, wy):
    """Plot X, Y and t vs. u (calculated using wx and wy)"""
    fig, axes = plt.subplots(constrained_layout=True, ncols=3, figsize=(9, 3), sharex=True, sharey=True)
    axes[0].scatter(X[:, 0], X[:, 1])
    axes[0].set(xlabel=xvars[0], ylabel=xvars[1], title='X')

    axes[1].scatter(Y[:, 0], Y[:, 1])
    axes[1].set(xlabel=yvars[0], ylabel=yvars[1], title='Y')

    axes[0].quiver(0, 0, wx[0][0], wx[1][0], color='black',
                   angles='xy', scale_units='xy', scale=0.25, width=0.015)
    
    axes[1].quiver(0, 0, wy[0][0], wy[1][0], color='red',
                   angles='xy', scale_units='xy', scale=0.25, width=0.015)

    
    
    t = X @ (wx / norm(wx)) 
    u = Y @ (wy / norm(wy))
    # Technical detail: we norm both w and q here to get similar reults from different methods
    
    cov = t.T @ u

    axes[2].scatter(t[:, 0], u[:, 0])
    axes[2].set(
        xlabel='X-scores (t)',
        ylabel='Y-scores (u)',
        title=f'Covariance: {cov[0][0]:.2f}')
    sns.despine(fig=fig)

In [ ]:
# Weights by hand:
wx = np.array([0.0, 1.0])
wx = wx.reshape(2, -1)  # Make it a column vector
    
wy = np.array([1.0, 1.0])
wy = wy.reshape(2, -1)  # Make it a column vector

print(f"wx.T = {wx.T}, shape of w: {wx.shape}")
print(f"wy.T = {wy.T}, shape of q: {wy.shape}")
make_plot(X, Y, wx, wy)

In the plot above, the X-data is projected onto the black vector and this gives the X-scores (t). In the
same way, the Y-data is projected onto the red vector and this gives the Y-scores.
The rightmost plot show the scores plotted against each other.

Here, we see that we can use the X-scores to predict the Y-scores. And we would probably make a good prediction.
But our aim is to predict the whole $\mathbf{Y}$! If we convert the Y-scores to $\mathbf{Y}$, we would probably
predict the molecular weight quite good, but fail at predicting the solubility.

Below, you will find the same code for creating $\mathbf{w}$ and $\mathbf{q}$ as above. Can you find
two other vectors that give a larger correlation?

In [ ]:
wx = np.array([0.0, 1.0])
wx = wx.reshape(2, -1)  # Make it a column vector
    
wy = np.array([0.0, 1.0])
wy = wy.reshape(2, -1)  # Make it a column vector

make_plot(X, Y, wx, wy)

## Making use of `PLSRegression` from `sklearn`

In [ ]:
model = PLSRegression(scale=False, n_components=2)
model.fit(X, Y)
make_plot(X, Y, model.x_weights_, model.y_weights_)

## Calculating the loadings

The covariance between $\mathbf{t}$ and $\mathbf{u}$ is


\begin{equation}
\mathbf{t}^\top \mathbf{u} = (\mathbf{X} \mathbf{w}_x)^\top (\mathbf{Y} \mathbf{w}_y)
= \mathbf{w}_x^\top \mathbf{X}^\top \mathbf{Y} \mathbf{w}_y
\end{equation}




is the same as finding the
[singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
of $\mathbf{X}^\top \mathbf{Y}$. Let us also try this:

In [ ]:
U, _, Vt = np.linalg.svd(X.T @ Y)  # Find the singular value decomposition.

wx = U[:, 0].reshape(2, -1)
wy = Vt[0, :].reshape(2, -1)
make_plot(X, Y, -wx, -wy)

In [ ]:
def simpls(X, Y, n_components=2):
    row_x, col_x = X.shape
    if Y.ndim == 1:
        Y = Y.reshape(-1, 1)
    row_y, col_y = Y.shape
    assert row_x == row_y
    
    Y0 = Y - np.mean(Y, axis=0)
    S = X.T @ Y0
    
    R = np.zeros((col_x, n_components))  # Weights
    P = np.zeros((col_x, n_components))  # Loadings
    T = np.zeros((row_x, n_components))  # Scores
        
    Q = np.zeros((col_y, n_components))  # Weights
    U = np.zeros((row_y, n_components))  # Loadings
    V = np.zeros((col_x, n_components))  # Scores


    for i in range(n_components):
        left, _, _ = np.linalg.svd(S.T @ S, full_matrices=False)
        wy = left[:, 0]#
        wy = wy.reshape(col_y, -1)
        r = S @ wy
        t = X @ r
        t = t - np.mean(t, axis=0)
        normt = norm(t)
        
        t /= normt
        r /= normt
        
        p = X.T @ t
        q = Y0.T @ t
        u = Y0 @ q
        v = np.copy(p)
        if i > 0:
            v = v - V @ (V.T @ p)
            u = u - T @ (T.T @ u)
        v /= norm(v)
        S = S - v @ (v.T @ S)
        R[:, i] = r.flatten()
        P[:, i] = p.flatten()
        Q[:, i] = q.flatten()
        T[:, i] = t.flatten()
        U[:, i] = u.flatten()
        V[:, i] = v.flatten()
        print(p.shape, r.shape)
    B = R @ Q.T
    h = np.diag(T @ T.T) + 1.0 / row_x
    varX = np.diag(P.T @ P) / (row_x - 1.0)
    varY = np.diag(Q.T @ Q) / (row_x - 1.0)
    return R, P, Q, T, U, V, B, h, varX, varY

In [ ]:
R, P, Q, T, U, V, B, h, varX, varY = simpls(X, Y)

In [ ]:
h

In [ ]:
wx = R[:, 0]
wy = Q[:, 0]
wx /= norm(wx)
wy /= norm(wy)
wx = wx.reshape(2, -1)
wy = wy.reshape(2, -1)
make_plot(X, Y, wx, wy)

In [ ]:
print(B / model.coef_)

In [ ]:
from sklearn.metrics import r2_score
Y_hat = X @ B
Y_hat2 = model.predict(X)
fig, axes = plt.subplots(constrained_layout=True, ncols=2, sharex=True, sharey=True)
axes[0].scatter(Y[:, 0], Y_hat[:, 0], label=f'R² = {r2_score(Y[:, 0], Y_hat[:, 0]):.3f}')
axes[0].scatter(Y[:, 1], Y_hat[:, 1], label=f'R² = {r2_score(Y[:, 1], Y_hat[:, 1]):.3f}')
axes[0].legend()
axes[1].scatter(Y[:, 0], Y_hat2[:, 0], label=f'R² = {r2_score(Y[:, 0], Y_hat2[:, 0]):.3f}')
axes[1].scatter(Y[:, 1], Y_hat2[:, 1], label=f'R² = {r2_score(Y[:, 1], Y_hat2[:, 1]):.3f}')
axes[1].legend()